Input: 
* https://www.kaggle.com/cguzman09/extracting-emails-from-enron-data-set
* https://medium.com/@raymond.m.yu/named-entity-recognition-using-the-enron-email-dataset-in-python-part-1-e56afd9d840b
* https://towardsdatascience.com/named-entity-recognition-using-the-enron-email-dataset-in-python-part-2-fb7ce69bc473

In [34]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output



In [35]:
df = pd.read_csv('data/emails.csv')

In [36]:
df

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...
...,...,...
517396,zufferli-j/sent_items/95.,Message-ID: <26807948.1075842029936.JavaMail.e...
517397,zufferli-j/sent_items/96.,Message-ID: <25835861.1075842029959.JavaMail.e...
517398,zufferli-j/sent_items/97.,Message-ID: <28979867.1075842029988.JavaMail.e...
517399,zufferli-j/sent_items/98.,Message-ID: <22052556.1075842030013.JavaMail.e...


In [37]:
df['message'][1]

"Message-ID: <15464986.1075855378456.JavaMail.evans@thyme>\nDate: Fri, 4 May 2001 13:51:00 -0700 (PDT)\nFrom: phillip.allen@enron.com\nTo: john.lavorato@enron.com\nSubject: Re:\nMime-Version: 1.0\nContent-Type: text/plain; charset=us-ascii\nContent-Transfer-Encoding: 7bit\nX-From: Phillip K Allen\nX-To: John J Lavorato <John J Lavorato/ENRON@enronXgate@ENRON>\nX-cc: \nX-bcc: \nX-Folder: \\Phillip_Allen_Jan2002_1\\Allen, Phillip K.\\'Sent Mail\nX-Origin: Allen-P\nX-FileName: pallen (Non-Privileged).pst\n\nTraveling to have a business meeting takes the fun out of the trip.  Especially if you have to prepare a presentation.  I would suggest holding the business plan meetings here then take a trip without any formal business meetings.  I would even try and get some honest opinions on whether a trip is even desired or necessary.\n\nAs far as the business meetings, I think it would be more productive to try and stimulate discussions across the different groups about what is working and what 

In [41]:
fromRegex = re.compile(r'From:\s(\w)+(\.)?(\w)*@(\w)+.com')
toRegex = re.compile(r'To:\s(\w)+(\.)?(\w)*@(\w)+.com')
m_f = fromRegex.search(df['message'][1])
m_e = toRegex.search(df['message'][1])
if m_f:
    print(m_f.group())
if m_e:
    print(m_e.group())

From: phillip.allen@enron.com
To: john.lavorato@enron.com


In [42]:
len(df)

517401

In [40]:
from_set = set()
for i in range(len(emails)):
    mf = fromRegex.search(emails['message'][i])
    if mf:
        if mf.group() not in from_set:
            from_set.add(mf.group())
to_set = set()
for i in range(len(emails)):
    me = toRegex.search(emails['message'][i])
    if me:
        if me.group() not in to_set:
            to_set.add(me.group())
            
print("From emails:", len(from_set))
print("To emails:", len(to_set))

From emails: 14452
To emails: 15354


In [43]:
docs = df.message.sample(1000)
allkeys = [re.findall('\n([\w\-]+):', doc[:doc.find('\n\n')]) for doc in docs]
allkeys = sum(allkeys,[])
allkeys = set(allkeys)
allkeys.add('Message-ID')
allkeys.add('Body')
allkeys

{'Bcc',
 'Body',
 'Cc',
 'Content-Transfer-Encoding',
 'Content-Type',
 'Date',
 'From',
 'Message-ID',
 'Mime-Version',
 'Subject',
 'To',
 'X-FileName',
 'X-Folder',
 'X-From',
 'X-Origin',
 'X-To',
 'X-bcc',
 'X-cc'}

In [44]:
def parse_doc(doc):
    keys = ['Message-ID']+re.findall('\n([\w\-]+):', doc[:doc.find('\n\n')])
    keys = pd.Series(keys).drop_duplicates().tolist()

    values = []
    for a, k in enumerate(keys):
        k = k+':'
        try:
            values.append(doc[doc.find(k)+len(k):doc.find(keys[a+1])].strip())
        except:
            values.append(doc[doc.find(k)+len(k):doc.find('\n\n')].strip())
    
    d = dict(zip(keys+['Body'],values+[doc[doc.find('\n\n'):].strip()]))
    k_to_remove = set(d.keys()) - set(allkeys)
    k_to_add = set(allkeys) - set(d.keys())
    
    for k in k_to_remove:
        d.pop(k)
    for k in k_to_add:
        d[k] = ''

    keys = [k[:-1] for k in keys]
    return d

In [81]:
#docs = df.message
docs = df.message.sample(2000)

In [82]:
parsed=[]
for doc in docs:
    parsed.append(parse_doc(doc))
    df_parsed = pd.DataFrame(parsed)

In [74]:
df_parsed.shape


(3000, 18)

In [79]:
df_parsed

,Message-ID,Date,From,To,Subject,Mime-Version,Content-Type,Content-Transfer-Encoding,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,Body,Bcc,Cc
0,<16882558.1075855056551.JavaMail.evans@thyme>,"Thu, 25 Oct 2001 02:27:51 -0700 (PDT)",master.amar@hoegh.no,"dan.masters@enron.com, tony.galt@enron.com, ji...",POSREP 25 OCT '01,1.0,text/plain; charset=ANSI_X3.4-1968,7bit,LNG/C Hoegh Galleon - Master <master.amar@hoeg...,"Masters, Dan </O=ENRON/OU=NA/CN=RECIPIENTS/CN=...",,,\PYBARBO (Non-Privileged)\Deleted Items,Ybarbo-P,PYBARBO (Non-Privileged).pst,HOEGH GALLEON\t25-Oct 12:00LT\t25-Oct 10:00(UT...,,
1,<30365155.1075858682952.JavaMail.evans@thyme>,"Mon, 29 Oct 2001 07:01:10 -0800 (PST)",jeff.dasovich@enron.com,"d..steffes@enron.com, susan.mara@enron.com, mi...",RE: Proposed SCE Negotiation Strategy,1.0,text/plain; charset=us-ascii,7bit,"Dasovich, Jeff </O=ENRON/OU=NA/CN=RECIPIENTS/C...","Steffes, James D. </O=ENRON/OU=NA/CN=RECIPIENT...","Shapiro, Richard </O=ENRON/OU=NA/CN=RECIPIENTS...",,"\Sanders, Richard B (Non-Privileged)\Sanders, ...",Sanders-R,"Sanders, Richard B (Non-Privileged).pst",There is currently a meeting set for this Thur...,"richard.shapiro@enron.com, b..sanders@enron.co...","richard.shapiro@enron.com, b..sanders@enron.co..."
2,<24185891.1075862459005.JavaMail.evans@thyme>,"Mon, 22 Oct 2001 09:49:02 -0700 (PDT)",paulo.issler@enron.com,"zimin.lu@enron.com, j.kaminski@enron.com, vasa...",FW: Material for review,1.0,text/plain; charset=us-ascii,7bit,"Issler, Paulo </O=ENRON/OU=NA/CN=RECIPIENTS/CN...","Lu, Zimin </O=ENRON/OU=NA/CN=RECIPIENTS/CN=Zlu...","McMichael Jr., Ed </O=ENRON/OU=NA/CN=RECIPIENT...",,"\VKAMINS (Non-Privileged)\Kaminski, Vince J\Inbox",Kaminski-V,VKAMINS (Non-Privileged).pst,Ed has written the draft for the first assignm...,ed.mcmichael@enron.com,ed.mcmichael@enron.com
3,<28270440.1075854584498.JavaMail.evans@thyme>,"Tue, 21 Nov 2000 13:12:00 -0800 (PST)",bryant@cheatsheets.net,cheatsheets@egroups.com,[cheatsheets] #138 Quick News (11-21-00),1.0,text/plain; charset=ANSI_X3.4-1968,7bit,"""Joe Bryant"" <bryant@cheatsheets.net>","""Cheatsheets Mailing List"" <cheatsheets@egroup...",,,\Eric_Bass_Dec2000\Notes Folders\All documents,Bass-E,ebass.nsf,"Hi Folks, \n?\nJust a few quick notes on this ...",,
4,<25810397.1075853177948.JavaMail.evans@thyme>,"Thu, 10 Aug 2000 12:40:00 -0700 (PDT)",matthias.lee@enron.com,britt.davis@enron.com,Re: D3605/In re M/V PACIFIC VIRGO - Summary of...,1.0,text/plain; charset=us-ascii,7bit,Matthias Lee,Britt Davis,"Janice R Moore, Alan Aronowitz, Harry M Collin...",,\Richard_Sanders_Oct2001\Notes Folders\All doc...,Sanders-R,rsanders.nsf,Dear Britt\n\nAttached below is a summary of t...,"janice.moore@enron.com, alan.aronowitz@enron.c...","janice.moore@enron.com, alan.aronowitz@enron.c..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,<18532918.1075848096189.JavaMail.evans@thyme>,"Fri, 25 Aug 2000 01:30:00 -0700 (PDT)",steven.kean@enron.com,,Skilling's office,1.0,text/plain; charset=us-ascii,7bit,Steven J Kean,,,,\Steven_Kean_June2001_4\Notes Folders\Discussi...,KEAN-S,skean.nsf,Skilling,,
2996,<9866783.1075851968468.JavaMail.evans@thyme>,"Mon, 18 Jun 2001 05:09:00 -0700 (PDT)",greg.mcmahon@enron.com,richard.shapiro@enron.com,Govt affairs Consultancy spend May Ytd,1.0,text/plain; charset=us-ascii,7bit,Greg McMahon,Richard Shapiro,Ginger Dernehl,,\Richard_Shapiro_Nov2001\Notes Folders\All doc...,SHAPIRO-R,rshapiro.nsf,"Rick, \n\nPlease see attached a summary on con...",ginger.dernehl@enron.com,ginger.dernehl@enron.com
2997,<12472032.1075854247126.JavaMail.evans@thyme>,"Sun, 4 Mar 2001 23:55:00 -0800 (PST)",megan.parker@enron.com,daren.farmer@enron.com,Tenaska IV Receivables,1.0,text/plain; charset=us-ascii,7bit,Megan Parker,Daren J Farmer,,,\Darren_Farmer_Jun2001\Notes Folders\Cleburne ...,Farmer-D,dfarmer.nsf,The email below is from James Armstrong regard...,,
2998,<19207390.1075854196835.JavaMail.evans@thyme>,"Tue, 27 Feb 2001 08:29:00 -0

In [75]:
df_parsed.resample('MS')['Message-ID'].count().plot(marker='o')

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'RangeIndex'

In [78]:
df_parsed.resample('MS')

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'RangeIndex'

In [77]:
df_parsed.index.month

AttributeError: 'RangeIndex' object has no attribute 'month'